# Generador de base para aplicar a recrédito

Este código genera una base de datos con el monto que se le puede ofertar a un cliente para que los ingresos por recrédito a n meses (con n dado y variable) sobrepasen los ingresos cancelados por recréditos y las reservas.

In [2]:
import pandas as pd
import pyodbc

## Conexión a los datos

In [5]:
# conectarse a la base de datos
def es_lectura_sql():
    # Establecer conexión
    conn = pyodbc.connect('DRIVER={SQL Server};'
                        'SERVER=BTBIDB-VM;'
                        'DATABASE=LNDDW;'
                        'Trusted_Connection=yes;')

    # Conectarse a datos de la cartera
    with open ("C:/Users/mariajose_chinchilla/Desktop/Queries/BTBIDBVM/evolucion_tasas.sql", "r") as q1:
        query1 = q1.read()
    # Leer datos con pandas
    cartera = pd.read_sql_query(query1, conn) 
    # Leer datos
    return cartera

In [4]:
df = pd.read_excel(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\db\cartera mayo 1 (1).xlsx")

## Programación de la desigualdad y funciones útliles

In [9]:
def desigualdad_recredito(alpha:float, ingresos_cancelados:float, tasa:float, plazo:int, n:int) -> float:
    factor1 = alpha + ingresos_cancelados
    if tasa > 1:
        tasa = tasa / 1200
    elif tasa <= 1:
        tasa = tasa / 12
    factor2 = ((n*tasa + 1 - (1 + tasa)**n)*((1+tasa)**plazo))/((1+tasa)**plazo-1) + (1+tasa)**n - 1 - 0.05*0.45
    return factor1 * factor2**-1

In [19]:
len(df[df["FECHA_VENCIMIENTO"] >= "2025-01-30"]["CODIGO_CLIENTE"].unique().tolist())

208809

In [27]:
len(df[(df["FECHA_APERTURA"] < "2024-01-01") & (df["FECHA_VENCIMIENTO"] >= "2024-07-30")]["CODIGO_CLIENTE"].unique().tolist())

194670